# QUIZ Множественная проверка гипотез

In [2]:
import pandas as pd
import numpy as np

from scipy.stats import pearsonr
from statsmodels.sandbox.stats.multicomp import multipletests

Task 1 Выберите задачи, в которых обязательно нужно применять поправку на множественную проверку гипотез

vСравнение эффективности лечения пациентов в подгруппах по большому количеству признаков

vЛокализация различий в активности мозга испытуемых в разных экспериментальных условиях

vПопарное сравнение средних большого количества выборок

vВыбор инвестиционных фондов с помощью сравнения доходности каждого из них с доходностью базового актива по нескольким историческим периодам

Task2 Классификатор C4.5 и три его модификации: с оптимизацией гиперпараметра m, гиперпараметра cf и с одновременной оптимизацией обоих гиперпараметров. Эти четыре классификатора сравнивались на 14 наборах данных. На каждом датасете был посчитан AUC каждого классификатора. Данные записаны в файле: AUCs

Используя критерий знаковых рангов, проведите попарное сравнение каждого классификатора с каждым. Выберите два классификатора, различие между которыми наиболее статистически значимо.

P.S.Для того чтобы построить пары классификаторов, воспользуйтесь функцией combinations из модуля itertools. Пример использования: combinations([a,b,c,d], 2)


Task 3:Сколько статистически значимых на уровне 0.05 различий мы обнаружили? 4

Task 4:Судя по данным из предыдущего опроса, настройка какого из параметров классификатора даёт более значимое увеличение качества?

m



In [5]:
aucs = pd.read_csv('AUCs.txt', sep = '\t', header = 0)

In [6]:
aucs.head()

,Unnamed: 0,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
0,adult (sample),0.763,0.768,0.771,0.798
1,breast cancer,0.599,0.591,0.590,0.569
2,breast cancer wisconsin,0.954,0.971,0.968,0.967
3,cmc,0.628,0.661,0.654,0.657
4,ionosphere,0.882,0.888,0.886,0.898


In [24]:
from itertools import combinations
#combinations()	p, r	r-length tuples, in sorted order, no repeated elements
#itertools.combinations(iterable, r)
#Return r length subsequences of elements from the input iterable.

In [62]:
from scipy import stats

In [46]:
#print (list(itertools.product(*aucs)))
print ("combinations example:")
p_value=[]
i = 1
for i_col,j_col in itertools.combinations(['C4.5','C4.5+m','C4.5+cf','C4.5+m+cf'],2):
    print (i_col,j_col,":",stats.wilcoxon(aucs[i_col] - aucs[j_col]))
    i += 1
    p_value.append(stats.wilcoxon(aucs[i_col] - aucs[j_col]).pvalue)
print(p_value)

combinations example:
C4.5 C4.5+m : WilcoxonResult(statistic=6.5, pvalue=0.01075713311978963)
C4.5 C4.5+cf : WilcoxonResult(statistic=43.0, pvalue=0.86126233009534803)
C4.5 C4.5+m+cf : WilcoxonResult(statistic=11.0, pvalue=0.015906444101703374)
C4.5+m C4.5+cf : WilcoxonResult(statistic=17.0, pvalue=0.046332729793395394)
C4.5+m C4.5+m+cf : WilcoxonResult(statistic=22.0, pvalue=0.32782567584464062)
C4.5+cf C4.5+m+cf : WilcoxonResult(statistic=10.0, pvalue=0.022909099354356588)
[0.01075713311978963, 0.86126233009534803, 0.015906444101703374, 0.046332729793395394, 0.32782567584464062, 0.022909099354356588]


In [96]:
index_min = np.argmin(p_value)
index_min

0

In [97]:
#второй способ
wilcoxon_array = np.array([])
for i, column1 in enumerate(aucs.columns):
    for j, column2 in enumerate(aucs.columns):
        if j > i >= 1:
            wilcoxon_array = np.append(wilcoxon_array,
                                       {stats.wilcoxon(aucs[column1], aucs[column2])[1]:column1+' '+column2})


In [86]:
print(type(wilcoxon_array))

<class 'numpy.ndarray'>


In [90]:
print(wilcoxon_array)

[{0.01075713311978963: 'C4.5 C4.5+m'} {0.86126233009534803: 'C4.5 C4.5+cf'}
 {0.015906444101703374: 'C4.5 C4.5+m+cf'}
 {0.046332729793395394: 'C4.5+m C4.5+cf'}
 {0.32782567584464062: 'C4.5+m C4.5+m+cf'}
 {0.022909099354356588: 'C4.5+cf C4.5+m+cf'}]


TASK 5. 
Сравнивая 4 классификатора между собой, мы проверили 6 гипотез. Давайте сделаем поправку на множественную проверку. Начнём с метода Холма. Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки этим методом? -0

# Поправка на множественную проверку

# Метод Холма

In [58]:
reject, p_corrected, a1, a2 = multipletests(p_value, 
                                            alpha = 0.05, 
                                            method = 'holm')
print(reject)
print(sum(reject))

[False False False False False False]
0


Первая поправка на множественную проверку гипотез, которой мы будем пользоваться, называется метод Холма. В рамках этой поправки мы ограничиваем вероятность того, что хотя бы на одном объекте будет ошибка, 5 %. Соответствующий метод реализован в библиотеке StatsModels, он называется multipletests, и для того чтобы передать внутрь набор наших p-value, мы их уже получили, также нужно передать уровень значимости, с которым мы работаем. Ну вот если мы хотим ограничить вероятность ошибки 5 %, тогда нам нужно передать уровень значимости 0,05. И также нужно указать метод, с помощью которого будет делаться правка. Мы используем метод Холма.

# Метод Бенджамини-Хохберга¶

Task 6 Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки методом Бенджамини-Хохберга?

Теперь давайте сделаем поправку на множественную проверку гипотез другим методом, например, давайте воспользуемся методом Бенджамини-Хохберга. Если в случае метода Холма мы с вами ограничивали вероятность сделать хотя бы одну ошибку, то в данном случае мы будем пытаться ограничить среднюю вероятность ошибок. Вот давайте ограничим среднюю вероятность ошибок 5 %, для этого нам снова придется использоваться коэффициент α = 0,05, и построим скорректированное значение p-value, а также снова рассчитаем, сколько же гипотез мы отвергнем с учетом данной правки.

In [60]:
reject, p_corrected, a1, a2 = multipletests(p_value, 
                                            alpha = 0.05, 
                                            method = 'fdr_bh')

In [61]:
print (sum(reject))

3


Для того чтобы это посчитать, нам снова придется воспользоваться той же самой функцией (функция multipletests из библиотеки StatsModels), однако в этом случае нам нужно передать ей метод ('fdr_bh'). 

Заменим столбцы p_corrected и rejected, которые мы с вами рассчитали предыдущим методом, на новые значения. Итак, заменяем и смотрим, как изменился DataFrame, как изменилась таблица. Структура осталась та же самая, единственное что скорректированные значения p-value и reject у нас теперь рассчитаны другим методом.

Насколько корректно, на ваш взгляд, применение метода Бенджамини-Хохберга в этой задаче?
некорректно
8. 
Мы подозреваем, что в проведённом с C4.5 эксперименте на самом деле классификаторы сильнее отличаются друг от друга, просто нам не удалось это заметить. Что можно сделать, чтобы увеличить вероятность обнаружения различий, если они действительно существуют?
Взять больше датасетов